# Project: Investigating No-Show Medical Appointments

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

#### Dataset: No-show Appointments

This dataset contains information from more than 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment. A number of characteristics about the patient are included in each row. Some of them are,

    1. ‘ScheduledDay’ tells us on what day the patient set up their appointment.
    2. ‘AppointmentDay’ tells us on what day the patient has to show up for the medical checkup.
    3. ‘Neighborhood’ indicates the location of the hospital.
    4. ‘Scholarship’ indicates whether or not the patient is enrolled in Brasilian welfare program Bolsa Família.
    5. Reasons include Hipertension, Diabetes, Alcoholism and Handcap
    6. ‘No-show’ says ‘No’ if the patient showed up to their appointment, and ‘Yes’ if they did not show up.
    
#### Below are the questions that will be explored over the course of the report.

    1. Does the difference between `scheduled_day` and `appointment_day` impact whether or not the patient shows up?
    2. Does day of week affect `no-show`?
    3. Does the `age_group` have a bearing on `no_show`?
    4. Which are the `neighbourhoods` with high number of appointments have higher `no_show` rate?
    5. Which condition or ailment ('hypertension', 'diabetes', 'alcoholism' and 'handcap') has higher `no-show` rate?

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

<a id='wrangling'></a>
## Data Wrangling

### General Properties

In [3]:
df = pd.read_csv('noshowappointments.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


Thankfully there are no null values and incorrect datatypes in the dataset.

In [4]:
df.head(3)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589980e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262960e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No


For easy remembrance of column names all column names are made lower case.

In [5]:
df.columns = [x.strip().lower() for x in df.columns]

In [8]:
df.head(1)

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No


### Data Cleaning

In [9]:
df.describe()

,patientid,appointmentid,age,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.920000e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172615e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173180e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439170e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999820e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


The dataset is mostly proper except for some records with age equals to -1, which is incorrect. Let's find those records and remove them.

In [10]:
df[df['age'] == -1]

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show
99832,4.659430e+14,5775010,F,2016-06-06T08:58:13Z,2016-06-06T00:00:00Z,-1,ROMÃO,0,0,0,0,0,0,No


Let's check if the above patientid has any other records so that we can replace age -1 with any other correct value.

In [11]:
df[df['patientid'] == 4.659430e+14]

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show
99832,4.659430e+14,5775010,F,2016-06-06T08:58:13Z,2016-06-06T00:00:00Z,-1,ROMÃO,0,0,0,0,0,0,No


So there is only one record with age -1, therefore let's remove that record.

In [12]:
df.drop([99832], inplace = True)
df[df['age'] == -1]

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show


Now let's check for duplicate records.

In [13]:
print(sum(df.duplicated()))
print(sum(df.patientid.duplicated()))
print(sum(df.appointmentid.duplicated()))

0
48783
0


So there are neither duplicate records nor duplicate appointment Ids, but there are 48783 duplicate patient IDs. This may be because a patient had to make multiple visits throughout the year. Moreover we still do not know if a patient ID represents just one patient or it can be shared by many. So, lets explore a combination four columns `patientid`, `gender`, `age` and `neighbourhood` to know if they together could refer one unique patient.

In [14]:
temp = df[['patientid', 'gender', 'age', 'neighbourhood']]
sum(temp.duplicated())

47061

There are so many duplicate combinations of `patientid`, `gender`, `age` and `neighbourhood`, it means many patients had multiple appointments throughout the year.

In [15]:
temp.drop_duplicates(inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
temp_2 = temp[temp.groupby(['patientid', 'gender', 'neighbourhood'])['age'].transform('nunique') > 1]
temp_2.head(2)

,patientid,gender,age,neighbourhood
40,9.968680e+14,F,55,TABUAZEIRO
73,4.542130e+13,F,68,REPÚBLICA


In [29]:
df[df.patientid.isin([4.542130e+13, 9.968680e+14])].sort_values('patientid')

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show
73,4.542130e+13,5552934,F,2016-04-06T18:12:38Z,2016-04-29T00:00:00Z,68,REPÚBLICA,0,1,1,0,0,1,No
84139,4.542130e+13,5704024,F,2016-05-16T14:54:47Z,2016-06-03T00:00:00Z,69,REPÚBLICA,0,1,1,0,0,1,No
40,9.968680e+14,5635881,F,2016-04-28T14:14:16Z,2016-04-29T00:00:00Z,55,TABUAZEIRO,0,0,0,0,0,0,No
100196,9.968680e+14,5616959,F,2016-04-25T15:06:32Z,2016-06-01T00:00:00Z,56,TABUAZEIRO,0,0,0,0,0,1,No


In [30]:
temp_3 = temp[temp.groupby(['patientid', 'age', 'neighbourhood'])['gender'].transform('nunique') > 1]
temp_3.head(2)

,patientid,gender,age,neighbourhood


In [31]:
temp_4 = temp[temp.groupby(['patientid', 'gender', 'age'])['neighbourhood'].transform('nunique') > 1]
temp_4.head(2)

,patientid,gender,age,neighbourhood
25920,6.326740e+12,M,0,BELA VISTA
34931,8.622320e+11,F,44,TABUAZEIRO


In [32]:
df[df.patientid.isin([6.326740e+12, 8.622320e+11])].sort_values('patientid')

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no-show
34931,8.622320e+11,5650308,F,2016-05-03T07:37:07Z,2016-05-03T00:00:00Z,44,TABUAZEIRO,0,0,0,0,0,0,No
62226,8.622320e+11,5549167,F,2016-04-06T09:31:06Z,2016-05-11T00:00:00Z,44,JABOUR,0,0,0,0,0,1,No
25920,6.326740e+12,5676283,M,2016-05-09T15:23:28Z,2016-05-11T00:00:00Z,0,BELA VISTA,0,0,0,0,0,0,No
52318,6.326740e+12,5742676,M,2016-05-30T08:14:29Z,2016-05-30T00:00:00Z,0,BELA VISTA,0,0,0,0,0,0,No
52675,6.326740e+12,5744244,M,2016-05-30T10:00:39Z,2016-05-30T00:00:00Z,0,SANTO ANDRÉ,0,0,0,0,0,0,No
95447,6.326740e+12,5760196,M,2016-06-01T14:26:47Z,2016-06-02T00:00:00Z,0,SANTO ANDRÉ,0,0,0,0,0,0,No
100692,6.326740e+12,5773163,M,2016-06-06T06:45:03Z,2016-06-07T00:00:00Z,0,SANTO ANDRÉ,0,0,0,0,0,0,No


The above results indicate a patient can be uniquely identified sometimes either by a combination of `patientid`, `gender` and `age` or by a combination of `patientid`, `gender` and `neighbourhood`. Because in some cases the patient's age had increased by one due to birthday and in some cases the patient had moved from one locality to another.

And patients with `age` equals to `zero` are assumed to be new borns.

This analysis was performed to just understand the data better, anyway we are not worried about an individual patient per se as far as the investigation of this dataset is concerned. However, our analysis will mainly be focused on finding some potential general factors that might be useful in determining if a patient will show up on the day of appointment or not.

<a id='eda'></a>
## Exploratory Data Analysis

### Research Question 1
Does the difference between `scheduled_date` and `appointment_date` impact whether or not the patient shows up?

First let's get the scheduled and appointment dates from `scheduledday` and `appointmentday` strings.

In [33]:
df['scheduled_date'] = df.scheduledday.str[:10]
df['scheduled_date'] = pd.to_datetime(df['scheduled_date'], format = '%Y-%m-%d')

df['appointment_date'] = df.appointmentday.str[:10]
df['appointment_date'] = pd.to_datetime(df['appointment_date'], format = '%Y-%m-%d')

In [ ]:
#Creating a new column for the difference between the two dates
df['sch_appt_diff'] = df['appointment_date'] - df['scheduled_date']